# Unit Testing with OUnit

```{note}
This section is a bit of a detour from our study of data types, but it's a good
place to take the detour: we now know just enough to understand how unit testing
can be done in OCaml, and there's no good reason to wait any longer to learn
about it.
```

Using the toplevel to test functions will only work for very small programs.
Larger programs need *test suites* that contain many *unit tests* and can be
re-run every time we update our code base. A unit test is a test of one small
piece of functionality in a program, such as an individual function.

We've now learned enough features of OCaml to see how to do unit testing with a
library called OUnit. It is a unit testing framework similar to JUnit in Java,
HUnit in Haskell, etc. The basic workflow for using OUnit is as follows:

* Write a function in a file `f.ml`. There could be many other functions in that
  file too.

* Write unit tests for that function in a separate file `f_test.ml`. That exact
  name, with an underscore and "test" is not actually essential, but is a
  convention we'll often follow.

* Build and run `f_test` to execute the unit tests.

The [OUnit documentation][ounitdoc] is available on Github.

[ounitdoc]: https://gildor478.github.io/ounit/ounit2/index.html

## An Example of OUnit

The following example shows you how to create an OUnit test suite. There are
some things in the example that might at first seem mysterious; they are
discussed in the next section.

Create a file named `sum.ml`, and put the following code into it:
```ocaml
let rec sum = function
  | [] -> 0
  | x :: xs -> x + sum xs
```

Now create a second file named `sum_test.ml`, and put this code into it:
```ocaml
open OUnit2
open Sum

let tests = "test suite for sum" >::: [
  "empty" >:: (fun _ -> assert_equal 0 (sum []));
  "singleton" >:: (fun _ -> assert_equal 1 (sum [1]));
  "two_elements" >:: (fun _ -> assert_equal 3 (sum [1; 2]));
]

let _ = run_test_tt_main tests
```

```{note}
Depending on what editor you are using, you might now see some errors about
OUnit2 and Sum. Ignore those for the moment: your code is correct, but your
editor doesn't understand it yet. We'll fix that later by creating a `.merlin`
file.
```

Finally, run these commands:

```console
$ ocamlbuild -pkgs oUnit sum_test.byte
$ ./sum_test.byte
```

You will get a response something like this:
```text
...
Ran: 3 tests in: 0.12 seconds.
OK
```

Now suppose we modify `sum.ml` to introduce a bug by changing the code
in it to the following:
```ocaml
let rec sum = function
  | [] -> 1 (* bug *)
  | x :: xs -> x + sum xs
```

If rebuild and re-execute `sum_test.byte`, all test cases now fail. The output
tells us the names of the failing cases. Here's the beginning of the output, in
which we've replaced some strings that will be dependent on your own local
computer with `...`:
```
FFF
==============================================================================
Error: test suite for sum:2:two_elements.

File ".../_build/oUnit-test suite for sum-...#01.log", line 8, characters 1-1:
Error: test suite for sum:2:two_elements (in the log).

Called from unknown location

not equal
------------------------------------------------------------------------------
```

The first line of that output
```
FFF
```
tells us that OUnit ran three test cases and all three <u>f</u>ailed.

The next interesting line
```
Error: test suite for sum:2:two_elements.
```
tells us that in the test suite named `test suite for sum` the test case at
index 2 named `two_elements` failed. The rest of the output for that test case
is not particularly interesting; let's ignore it for now.

## Explanation of the OUnit Example

Let's study more carefully what we just did in the previous section.
We had a source file named `sum.ml` with this code:
```
let rec sum = function
  | []    -> 0
  | x::xs -> x + sum xs
```

And a test file named `sum_test.ml` with this code:
```
open OUnit2
open Sum

let tests = "test suite for sum" >::: [
  "empty"  >:: (fun _ -> assert_equal 0 (sum []));
  "one"    >:: (fun _ -> assert_equal 1 (sum [1]));
  "onetwo" >:: (fun _ -> assert_equal 3 (sum [1; 2]));
]

let _ = run_test_tt_main tests
```

In the test file,
`open OUnit2` brings into scope the many definitions in OUnit2, which is version 2
of the OUnit framework.  And `open Sum` brings into scope the definitions from
`sum.ml`.  We'll learn more about scope and the `open` keyword later in the course.

Then we created a list of test cases:
```
[
  "empty"  >:: (fun _ -> assert_equal 0 (sum []));
  "one"    >:: (fun _ -> assert_equal 1 (sum [1]));
  "onetwo" >:: (fun _ -> assert_equal 3 (sum [1; 2]));
]
```
Each line of code is a separate test case.  A test case has a string giving it a
descriptive name, and a function to run as the test case.  In between the name
and the function we write `>::`, which is a custom operator defined by the OUnit
framework.  Let's look at the first function from above:
```
fun _ -> assert_equal 0 (sum [])
```
Every test case function receives as input a parameter that OUnit calls a *test context*.
Here (and in many of the test cases we write) we don't actually need to worry about
the context, so we use the underscore to indicate that the function ignores its input.
The function then calls `assert_equal`, which is a function provided by OUnit that
checks to see whether its two arguments are equal.  If so the test case succeeds.
If not, the test case fails.

Then we created a test suite:
```
let tests = "test suite for sum" >::: [
  "empty"  >:: (fun _ -> assert_equal 0 (sum []));
  "one"    >:: (fun _ -> assert_equal 1 (sum [1]));
  "onetwo" >:: (fun _ -> assert_equal 3 (sum [1; 2]));
]
```
The `>:::` operator is another custom OUnit operator.  It goes between the name
of the test suite and the list of test cases in that suite.

Then we ran the test suite:
```
let _ = run_test_tt_main tests
```

The function `run_test_tt_main` is provided by OUnit.  It runs a test suite and
prints the results of which test cases passed vs. which failed to standard output.
The use of underscore here indicates that we don't care what value the function
returns; it just gets discarded.

Finally, when we compiled the test file, we linked in the OUnit package, which
has slightly unusual capitalization (which some platforms care about and others
are agnostic about):
```
$ ocamlbuild -pkgs oUnit sum_test.byte
```
If you get tired of typing the `pkgs oUnit` part of that, you can instead create
a file named `_tags` (note the underscore) in the same directory and put
the following into it:
```
true: package(oUnit)
```
Now Ocamlbuild will automatically link in OUnit everytime you compile in this
directory, without you having to give the `pkgs` flag. The tradeoff is that
you now have to pass a different flag to Ocamlbuild:
```
$ ocamlbuild -use-ocamlfind sum_test.byte
```
And you will continue having to pass that flag as long as the `_tags` file exists.
Why is this any better?  If there are many packages you want to link, with
the tags file you end up having to pass only one option on the command
line, instead of many.
## Improving OUnit Output

In our example with the buggy implementation of `sum`,
we got the following output:

```
==============================================================================
Error: test suite for sum:2:onetwo.

File ".../_build/oUnit-test suite for sum-...#01.log", line 8, characters 1-1:
Error: test suite for sum:2:onetwo (in the log).

Called from unknown location

not equal
------------------------------------------------------------------------------
```

Let's see how to improve that output to be a little more informative.

### Stack traces

The `Called from an unknown location` indicates OCaml was unable
to provide a stack trace.  That happened because, by default,
stack traces are disabled.  We can enable them by compiling
the code with the debug tag:

```
$ ocamlbuild -pkgs oUnit -tag debug sum_test.byte
$ ./sum_test.byte

==============================================================================
Error: test suite for sum:2:onetwo.

File "/Users/clarkson/tmp/sum/_build/oUnit-test suite for sum-...#01.log", line 9, characters 1-1:
Error: test suite for sum:2:onetwo (in the log).

Raised at file "src/oUnitAssert.ml", line 45, characters 8-27
Called from file "src/oUnitRunner.ml", line 46, characters 13-26

not equal
------------------------------------------------------------------------------
```

Now we see the stack trace that resulted from `assert_equal` raising an
exception.  You'll probably agree that stack trace isn't very
informative though:  what matters is which test case fails, not which
files in the implementation of OUnit were involved in raising the
exception.  And we could already identify the failing test case from the
first line of output.  (It's the test case named `onetwo`, which at
position 2 in the test suite named `test suite for sum`.)

So we don't usually bother enabling stack traces for OUnit test suites.
Nonetheless, it could occasionally be useful if your *own* code is
raising exceptions that you want to track down.

### Output values

The `not equal` in the OUnit output means that `assert_equal` discovered
the two values passed to it in that test case were not equal.  That's
not so informative:  we'd like to know *why* they're not equal.
In particular, we'd like to know what the actual output
produced by `sum` was for that test case.  To find out,
we need to pass an additional argument to `assert_equal`.
That argument, whose label is `printer`, should be a function
that can transform the outputs to strings.  In this case, the
outputs are integers, so `string_of_int` from the Stdlib
module will suffice.  We modify the test suite as follows:

```
let tests = "test suite for sum" >::: [
  "empty"  >:: (fun _ -> assert_equal 0 (sum []) ~printer:string_of_int);
  "one"    >:: (fun _ -> assert_equal 1 (sum [1]) ~printer:string_of_int);
  "onetwo" >:: (fun _ -> assert_equal 3 (sum [1; 2]) ~printer:string_of_int);
]
```

And now we get more informative output:
```
==============================================================================
Error: test suite for sum:2:onetwo.

File "/Users/clarkson/tmp/sum/_build/oUnit-test suite for sum-sauternes#01.log", line 8, characters 1-1:
Error: test suite for sum:2:onetwo (in the log).

Called from unknown location

expected: 3 but got: 4
------------------------------------------------------------------------------
```

That output means that the test named `onetwo` asserted the equality
of `3` and `4`.  The expected output was `3` because that was the
first input to `assert_equal`, and that function's specification
says that in `assert_equal x y`, the output you (as the tester)
are expecting to get should be `x`, and the output the function
being tested actually produces should be `y`.

Notice how our test suite is accumulating a lot of redundant code.
In particular, we had to add the `printer` argument to several
lines.  Let's improve that code by factoring out a function
that constructs test cases:

```
let make_sum_test name expected_output input =
  name >:: (fun _ -> assert_equal expected_output (sum input) ~printer:string_of_int)

let tests = "test suite for sum" >::: [
  make_sum_test "empty" 0 [];
  make_sum_test "one" 1 [1];
  make_sum_test "onetwo" 3 [1; 2];
]
```

For output types that are more complicated than integers, you will
end up needing to write your own functions to pass to `printer`.
This is similar to writing `toString()` methods in Java: for
complicated types you invent yourself, the language doesn't know
how to render them as strings.  You have to provide the code
that does it.
## OUnit and Files

Creating a couple more files can make the use of OUnit more pleasant.

### Tags file

We compiled the OUnit test file, we had to specify linking of OUnit:
```
$ ocamlbuild -pkgs oUnit sum_test.byte
```
If you get tired of typing the `pkgs oUnit` part of that, you can instead create
a file named `_tags` (note the underscore) in the same directory and put
the following into it:
```
true: package(oUnit)
```
Now Ocamlbuild will automatically link in OUnit everytime you compile in this
directory, without you having to give the `pkgs` flag. The tradeoff is that
you now have to pass a different flag to Ocamlbuild:
```
$ ocamlbuild -use-ocamlfind sum_test.byte
```
And you will continue having to pass that flag as long as the `_tags` file exists.
Why is this any better?  If there are many packages you want to link, with
the tags file you end up having to pass only one option on the command
line, instead of many.

### Merlin file

If you are using Merlin (e.g., your editor is VS Code or Emacs), you will notice
two things that aren't quite optimal at this point.  First, Merlin doesn't
understand the OUnit code.  Second, Merlin doesn't understand the code located
in `sum.ml` while you are editing `sum_test.ml`.  To fix both of those problems,
create a file in the same directory and name it `.merlin`.  (Note the leading
dot in that filename.)  In that file put the following:
```
B _build
PKG oUnit
```
The first line tells Merlin to look for compiled code from other source files
inside the `_build` directory, which is where Ocamlbuild places compiled code.
The second line tells Merlin to look for the oUnit package.  You might need
to restart your editor for these changes to take effect.  After that, you definitely
have to compile the code with Ocamlbuild, so that the compiled code exists in
the `_build` directory where Merlin now expects to find it.

## Test-driven Development

Testing doesn't have to happen strictly after you write code. In
*test-driven development* (TDD), testing comes first! It emphasizes
*incremental* development of code: there is always something that can be
tested. Testing is not something that happens after implementation;
instead, *continuous testing* is used to catch errors early. Thus, it is
important to develop unit tests immediately when the code is written.
Automating test suites is crucial so that continuous testing requires
essentially no effort.

Here's an example of TDD.  We deliberately choose an exceedingly simple
function to implement, so that the process is clear.  Suppose we are
working with a datatype for days:

    type day = Sunday | Monday | Tuesday | Wednesday
             | Thursday | Friday | Saturday

And we want to write a function `next_weekday : day -> day` that returns
the next weekday after a given day. We start by writing the most basic,
broken version of that function we can:

    let next_weekday d = failwith "Unimplemented"

Then we write the simplest unit test we can imagine. For example,
we know that the next weekday after Monday is Tuesday. So we add a test:

```
let tests = "test suite for next_weekday" >::: [
  "tue_after_mon"  >:: (fun _ -> assert_equal (next_weekday Monday) Tuesday);
]
```

Then we run the OUnit test suite.  It fails, as expected.  That's good!
Now we have a concrete goal, to make that unit test pass. We revise
`next_weekday` to make that happen:

    let next_weekday d =
      match d with
      | Monday -> Tuesday
      | _ -> failwith "Unimplemented"

We compile and run the test; it passes. Time to add some more tests.
The simplest remaining possibilities are tests involving just weekdays,
rather than weekends.  So let's add tests for weekdays.

```
let tests = "test suite for next_weekday" >::: [
  "tue_after_mon"  >:: (fun _ -> assert_equal (next_weekday Monday) Tuesday);
  "wed_after_tue"  >:: (fun _ -> assert_equal (next_weekday Tuesday) Wednesday);
  "thu_after_wed"  >:: (fun _ -> assert_equal (next_weekday Wednesday) Thursday);
  "fri_after_thu"  >:: (fun _ -> assert_equal (next_weekday Thursday) Friday);
]
```

We compile and run the tests; many fail. That's good! We add new
functionality:

    let next_weekday d =
      match d with
      | Monday -> Tuesday
      | Tuesday -> Wednesday
      | Wednesday -> Thursday
      | Thursday -> Friday
      | _ -> failwith "Unimplemented"

We compile and run the tests; they pass.  At this point we could move
on to handling weekends, but we should first notice something about
the tests we've written:  they involve repeating a lot of code.
In fact, we probably wrote them by copying-and-pasting the first
test, then modifying it for the next three.  That's a sign that
we should *refactor* the code.  (As we did before with the `sum`
function we were testing.)

Let's abstract a function that creates test cases for `next_weekday`:

```
let make_next_weekday_test name expected_output input=
  name >:: (fun _ -> assert_equal expected_output (next_weekday input))

let tests = "test suite for next_weekday" >::: [
  make_next_weekday_test "tue_after_mon" Tuesday Monday;
  make_next_weekday_test "wed_after_tue" Wednesday Tuesday;
  make_next_weekday_test "thu_after_wed" Thursday Wednesday;
  make_next_weekday_test "fri_after_thu" Friday Thursday;
]
```

Now we finish the testing and implementation by handling weekends.
First we add some test cases:
```
  ...
  make_next_weekday_test "mon_after_fri" Monday Friday;
  make_next_weekday_test "mon_after_sat" Monday Saturday;
  make_next_weekday_test "mon_after_sun" Monday Sunday;
  ...
```

Then we finish the function:

```
let next_weekday =
  match d with
  | Monday -> Tuesday
  | Tuesday -> Wednesday
  | Wednesday -> Thursday
  | Thursday -> Friday
  | Friday -> Monday
  | Saturday -> Monday
  | Sunday -> Monday
```

Of course, most people could write that function without errors
even if they didn't use TDD.  But rarely do we implement functions
that are so simple.

**Process.** Let's review the process of TDD:

- Write a failing unit test case.
  Run the test suite to prove that the test case fails.
- Implement just enough functionality to make the test case pass.
  Run the test suite to prove that the test case passes.
- Improve code as needed.  In the example above we refactored
  the test suite, but often we'll need to refactor the
  functionality being implemented.
- Repeat until you are satisfied that the test suite provides
  evidence that your implementation is correct.